In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
#Helper functions

def norm_pdf(x, mu, sig):
    #Returns the PDF of Gaussian distribution (for plotting)
    #with mean=mu
    #and standard deviation = sig
    p = (1.0/np.sqrt(2*np.pi*sig**2))*np.exp(-(x-mu)**2/(2*sig**2))
    return p

def data(mu1 = -1.0, mu2 = 2.0, sig1 = 0.5, sig2 = 0.6, batch_size = 1024):
    #Multimodel distribution
    s1 = np.random.normal(mu1,sig1,[batch_size//2,1])
    s2 = np.random.normal(mu2,sig2,[batch_size//2,1])
    d = np.concatenate((s1,s2),axis=0).reshape([batch_size,1])
    np.random.shuffle(d)
    return d

def generate_image(iteration, x_fake, mu1 = -1.0, mu2 = 2.0, sig1 = 0.5, sig2 = 0.6):
            xplot = np.linspace(-4,4,1000)
            p1 = norm_pdf(xplot,mu1,sig1)
            p2 = norm_pdf(xplot,mu2,sig2) 
            p = p1/2+p2/2
        
            plt.clf()
            plt.hist(x_fake,100,density=True)
            plt.plot(xplot,p,'g-')
            plt.xlim(-4,4)
            plt.xlabel("x")
            plt.ylabel("Density")
            plt.title("Step = {}".format(iteration))
            plt.show()

In [ ]:
def generator(Z, output_dim = 1, h_dim = 32, reuse = False):
    with tf.variable_scope('generator',reuse=reuse):
        hidden1 = tf.layers.dense(Z,h_dim)
        hidden1 = tf.nn.relu(hidden1)
        
        hidden2 = tf.layers.dense(hidden1,h_dim)
        hidden2 = tf.nn.relu(hidden2)
        
        hidden3 = tf.layers.dense(hidden2,h_dim)
        hidden3 = tf.nn.relu(hidden3)
        
        output = tf.layers.dense(hidden3,output_dim)
        return output        

In [ ]:
def discriminator(X, h_dim = 32, reuse = False):
    with tf.variable_scope('discriminator',reuse=reuse):
        hidden1 = tf.layers.dense(X,h_dim)
        hidden1 = tf.nn.relu(hidden1)
        
        hidden2 = tf.layers.dense(hidden1,h_dim)
        hidden2 = tf.nn.relu(hidden2)
        
        hidden3 = tf.layers.dense(hidden2,h_dim)
        hidden3 = tf.nn.relu(hidden3)
        
        logits = tf.layers.dense(hidden3,1)
        return logits 

In [ ]:
def generator_loss(logits_fake):
    labels = tf.ones_like(logits_fake)
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits_fake)
    return tf.reduce_mean(loss)

def discriminator_loss(logits_real, logits_fake):
    labels_real = tf.ones_like(logits_real)
    loss_real = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_real, logits=logits_real)
    
    labels_fake = tf.zeros_like(logits_fake)
    loss_fake = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_fake, logits=logits_fake)
    return tf.reduce_mean(loss_real) + tf.reduce_mean(loss_fake)

In [ ]:
def training_op(disc_loss, gen_loss, learning_rate = 1e-4, beta1 = 0.0, beta2 = 0.9):
    disc_var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='discriminator')
    gen_var_list=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,scope='generator')
    
    disc_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=beta1,beta2=beta2)
    gen_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=beta1,beta2=beta2)
    
    disc_train_op = disc_optimizer.minimize(disc_loss,var_list=disc_var_list)
    gen_train_op = disc_optimizer.minimize(gen_loss,var_list=gen_var_list)
    return disc_train_op,gen_train_op

In [ ]:
def input_data(z_dim = 2, output_dim = 1, batch_size = 1024):
    X = tf.placeholder(dtype=tf.float32, shape = [batch_size, output_dim])
    Z = tf.random_normal([batch_size, z_dim])
    return X,Z

In [ ]:
#Hyperparmetrs
h_dim = 32
z_dim = 2
output_dim = 1
batch_size = 1024
learning_rate = 1e-4
beta1 = 0.0
beta2 = 0.9
epochs = 201
batch_it = 10

#distribution parameters
mu1 = -1.0
mu2 = 2.0  
sig1 = 0.5
sig2 = 0.6

In [ ]:
g1 = tf.Graph()
with g1.as_default() as graph:
    X,Z = input_data(z_dim, output_dim, batch_size)
    X_fake = generator(Z, output_dim=output_dim, h_dim=h_dim)

    logits_real = discriminator(X,h_dim=h_dim)
    logits_fake = discriminator(X_fake,h_dim=h_dim,reuse=True)

    disc_loss = discriminator_loss(logits_real,logits_fake)
    gen_loss = generator_loss(logits_fake)

    disc_train_op,gen_train_op = training_op(disc_loss, gen_loss, learning_rate=learning_rate, beta1=beta1, beta2=beta2)

In [ ]:
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(epochs):
        d = data(mu1=mu1,mu2=mu2,sig1=sig1,sig2=sig2,batch_size=batch_size) 
        feed_dict = {X:d}
        for i in range(batch_it):
            _,disc_cost,_,gen_cost = sess.run([disc_train_op,disc_loss,gen_train_op,gen_loss],feed_dict=feed_dict)
            _ = sess.run(gen_train_op)
        if epoch %20 == 0:
            x_fake = sess.run(X_fake)
            print("====")
            print("Iteration: {}/{}".format(epoch,epochs-1))
            print("Discriminator loss: {}".format(disc_cost))
            print("Generator loss: {}".format(gen_cost))
            generate_image(epoch,x_fake)